# TSFresh with 50% Overlap:

### for EMG Dataset: WS=200
### for Non-EMG Dataset: WS=55

### Used Dataset:imu-dataset-original

In [ ]:
## Function to include time and id columns in the data:
def id_time(df,step):
    
    #Removing unwanted columns.
    df.drop(columns=[i for i in df.columns if "timestamp" in i],inplace=True)
        
    #Sliding Window with 50% overlap:
    w,t=step,(step//2)
    r = np.arange(len(df))  # creating an array
    s = r[::t]              #selecting elements with a step of t, i.e. half of SL
    z = list(zip(s, s + w)) #Creating a list of tuples, each tuple holding the starting and ending row numbers.
    g = lambda t: df.iloc[t[0]:t[1]]
    j=pd.concat(map(g, z))
    
    j["id"]=0
    j["time"]=0 
    
    l=np.arange(0,len(j),step=step)
    l=np.append(l,len(j))
    
    j.reset_index(drop=True,inplace=True)
    
    #Sliding Window:    
    for i in range(len(l)-1):
        time = np.arange(len(j[l[i]:l[i+1]]))
        t_id=np.full(len(time),i)
        j.iloc[l[i]:l[i+1],list(j.columns).index("time")]=time
        j.iloc[l[i]:l[i+1],list(j.columns).index("id")]=t_id
                
    return j

## Function to extract time series features from the data:
def funct_tsfresh(df):
    #Extracting Features
    df_extracted = extract_features(df, column_id="id", column_sort="time")
    
    #Imputing the NaNs
    df_extracted=impute(df_extracted)
    
    return df_extracted

In [ ]:
## Function to complete the preprocessing (combining files) and feature extraction process
def funct_preprocessing(l,f,fn):
    
    emg=pd.DataFrame(columns=['timestamp', 'emg1', 'emg2', 'emg3', 'emg4', 'emg5', 'emg6', 'emg7',
       'emg8'])
    acc=pd.DataFrame()
    gyro=pd.DataFrame()
    euler=pd.DataFrame()
    ori=pd.DataFrame()
    
    for i in sorted(os.listdir(os.path.join(dir,l))):
        
        if "emg" in i:
            emg=pd.concat([emg,pd.read_csv(os.path.join(dir,l,i))],axis=0)
            
        elif "acc" in i or "gyro" in i or "orientation" in i:  
        #### accelerometer,gyro,orientation have same feature names. Hence changing the column names
            if "acc" in i:
                temp=pd.read_csv(os.path.join(dir,l,i))
                acc=pd.concat([acc,temp])

            elif "gyro" in i:
                temp=pd.read_csv(os.path.join(dir,l,i))
                gyro=pd.concat([gyro,temp])
                
            elif "Euler" in i:
                temp=pd.read_csv(os.path.join(dir,l,i))
                euler=pd.concat([euler,temp])
                
            else:
                temp=pd.read_csv(os.path.join(dir,l,i))
                ori=pd.concat([ori,temp])

    acc.columns=["timestamp","x_acc","y_acc","z_acc"]
    acc.drop("timestamp",axis=1,inplace=True)
    gyro.columns=["timestamp","x_gyro","y_gyro","z_gyro"]
    gyro.drop("timestamp",axis=1,inplace=True)
    ori.columns=["timestamp","x_ori","y_ori","z_ori","w"]
    ori.drop("timestamp",axis=1,inplace=True)
    
    nemg=pd.concat([acc,gyro,ori,euler],axis=1)
    
    emg.reset_index(drop=True,inplace=True)
    nemg.reset_index(drop=True,inplace=True)
    
    type_emg={"emg1":float,
          "emg2":float,
          "emg3":float,
          "emg4":float,
          "emg5":float,
          "emg6":float,
          "emg7":float,
          "emg8":float
    }
    emg=emg.astype(type_emg)
    
###Now comes the TSFresh Part(Unsupervised)

    emg=id_time(emg,200)
    emg=funct_tsfresh(emg)
    nemg=id_time(nemg,55)
    nemg=funct_tsfresh(nemg)    
    
    emg["label"]=l
    nemg["label"]=l
    
    f=pd.concat([f,emg],axis=0)
    fn=pd.concat([fn,nemg],axis=0)
    
    return f,fn

In [ ]:
## To save time:
start=X   #### X is the custom value, like 18-21
end=X+3
f=pd.DataFrame()
fn=pd.DataFrame()
for i in list(string.ascii_uppercase)[start:end]:
    f,fn=funct_preprocessing(i,f,fn)

In [ ]:
f.to_csv(f"IMU_EMG_AfterTSFresh_{start}-{end}.csv",index=False)
fn.to_csv(f"IMU_NonEMG_AfterTSFresh_{start}-{end}.csv",index=False)